## 台北市房價分析
   
   # 資料預處理

   檢查資料是否有重複的可能性以及遺失值

   坪數與房價價格轉換

       1、 單價(萬/坪): 單價元平方公尺乘上3.305並除10000
   
       2、 (建物、土地、主建物、附屬建物、陽台面積、車位)的坪數 (3.305平方公尺) 對除

       3、 (房屋土地及車位)的總價元除上10000,得到萬元為單位的價格

   屋齡資料上的轉換

       1、 轉換資料到適合計算屋齡的時間資料格式

       2、 交易資料跟建築完成年月日資料對減獲得屋齡

       3、 對屋齡除上 365 天
       
       4、 屋齡為負則化為0 ,在另新變數為是否有購買預售屋(1 : 代表有購買預售屋 , 0 : 未購買)

   
     轉換 轉移樓層
          
          1、總樓層資料換成數字 

          2、 找出轉移層次中的騎樓、夾層、地下室、平台、多樓層等交易細節，轉為屬質變數

          3、轉移層次中的全轉為透天厝，其餘轉為數字

     計算面積占比 包含(主建物、附屬建物、陽台面積以及公占比)

     轉換備註資料(包含是否為特殊關係交易、違章建物、未登記建物以及公共設施)





In [239]:
import pandas as pd
data112 = pd.read_excel(r'C:\資料探勘\經緯度資料標記\112_label_.xlsx',index_col=False)

## 確認資料是否有重複


In [240]:
print(f'資料重複筆數: {data112[data112.duplicated()].shape[0]}')

資料重複筆數: 490


In [241]:
# REMOVE DUPLICATE DATA SET 

data112= data112[- data112.duplicated()]

## 檢查資料是否有重複的可能以及遺失值

解釋 : 
       1、 非都市土地用地資料只有一筆資料，占比很小，因此列為不考慮的變數

       2、 都市土地使用分區遺失資料數為 563 ,推測可能為是沒有土地的房屋

       3、 建築完成年月日遺失值，推測可能是因為房子在內政部開始登記前建造，所以有部分資料蒐集不到

       4、 總樓層、移轉層次推測是為只有土地跟車位交易

       5、 單價元平方公尺 : 推測可能是車位

In [242]:
print(f'資料列數 : {data112.shape[0]}' ,f'資料行數 : {data112.shape[1]}')
print('=======================================')

print('資料遺失情況')
print(data112.isna().sum())

資料列數 : 19986 資料行數 : 37
資料遺失情況
鄉鎮市區               0
交易標的               0
土地位置建物門牌           0
土地移轉總面積平方公尺        0
都市土地使用分區         426
非都市土地使用分區      19985
非都市土地使用編定      19986
交易年月日              0
交易筆棟數              0
移轉層次             184
總樓層數             202
建物型態               0
主要用途             749
主要建材             184
建築完成年月           904
建物移轉總面積平方公尺        0
建物現況格局.房           0
建物現況格局.廳           0
建物現況格局.衛           0
建物現況格局.隔間          0
有無管理組織             0
總價元                0
單價元平方公尺         1022
車位類別           13777
車位移轉總面積平方公尺        0
車位總價元              0
備註              6525
編號                 0
主建物面積              0
附屬建物面積             0
陽台面積               0
電梯                 0
移轉編號           13210
m                  0
n                  0
經度               148
緯度               148
dtype: int64


. 調查移轉層數遺失原因

In [243]:
print(f'移轉層次遺失次數 : {data112[data112['移轉層次'].isna()].shape}')

print(f'交易標的只有土地及車位以及移轉層次的遺失筆數 : {data112.query(('((交易標的 == "車位") | (交易標的 == "土地")) & @pd.isna(移轉層次)')).shape[0]}')

移轉層次遺失次數 : (184, 37)
交易標的只有土地及車位以及移轉層次的遺失筆數 : 182


In [244]:
print(data112['交易標的'].value_counts())

交易標的
房地(土地+建物)       13643
房地(土地+建物)+車位     5174
車位               1022
建物                147
Name: count, dtype: int64


## 原因是建物含有遺失值導致

In [245]:
pd.set_option("display.max_columns",None)
print(data112.query('(交易標的 == "建物") & @pd.isna(移轉層次)'))

      鄉鎮市區 交易標的              土地位置建物門牌  土地移轉總面積平方公尺 都市土地使用分區 非都市土地使用分區  \
7559   內湖區   建物  臺北市內湖區麗山街３４８巷１號等共同使用          0.0      NaN       NaN   
17761  內湖區   建物  臺北市內湖區行善路４５０號五樓等共有部分          0.0      NaN       NaN   

       非都市土地使用編定    交易年月日      交易筆棟數 移轉層次 總樓層數           建物型態 主要用途 主要建材  \
7559         NaN  1120731  土地0建物1車位0  NaN  NaN   公寓(5樓含以下無電梯)  NaN  NaN   
17761        NaN  1121116  土地0建物1車位0  NaN  NaN  華廈(10層含以下有電梯)  NaN  NaN   

       建築完成年月  建物移轉總面積平方公尺  建物現況格局.房  建物現況格局.廳  建物現況格局.衛 建物現況格局.隔間 有無管理組織  \
7559      NaN        99.28         0         0         0         無      無   
17761     NaN        13.15         0         0         0         無      有   

           總價元   單價元平方公尺 車位類別  車位移轉總面積平方公尺  車位總價元  \
7559   3600000   36261.0  NaN          0.0      0   
17761  2189000  166464.0  NaN          0.0      0   

                                   備註                   編號  主建物面積  附屬建物面積  \
7559            親友、員工、共有人或其他特殊關係間之交易；  RPPNMLKMQHIGFCA57EA    0.0     0.0   
17761  本

# 對平方公尺單價元遺失原因是為車位價格分開而因此無法計算

In [246]:
print(f"平方公尺單價元遺失次數 : {data112['單價元平方公尺'].isna().sum()} ")
print(f'平方公尺單價元跟交易標的為車位遺失次數 : {data112.query(('(交易標的 == "車位")  & @pd.isna(單價元平方公尺)')).shape[0]}')


平方公尺單價元遺失次數 : 1022 
平方公尺單價元跟交易標的為車位遺失次數 : 1022


## 坪數與房價價格轉換

In [247]:
data112['單價(萬/坪)'] = data112['單價元平方公尺']*3.305/10000

data112['土地移轉總面積(坪)'] = data112['土地移轉總面積平方公尺']/3.305
data112['建物移轉總面積(坪)'] = data112['建物移轉總面積平方公尺']/3.305
data112['車位移轉總面積(坪)'] = data112['車位移轉總面積平方公尺']/3.305
data112['主建物面積(坪)'] = data112['主建物面積']/3.305
data112['附屬建物面積(坪)'] = data112['附屬建物面積']/3.305
data112['陽台面積(坪)'] = data112['陽台面積']/3.305

data112['車位總價元(萬)'] = data112['車位總價元']/10000
data112['總價元(萬)'] = data112['總價元']/10000




## 計算屋齡
   
    1、轉換資料到適合計算屋齡的時間資料格式

    2、交易資料跟建築完成年月日資料對減獲得屋齡

    3、對屋齡除上 365 天

    4、屋齡為負則化為0 ,在另新變數為是否有購買預售屋(1 : 代表有購買預售屋 , 0 : 未購買)

    

In [248]:
import math

def transfor_date(date_str):
    try:

        date_str = str(int(float(date_str)))
        date = int(date_str[-2:])
        month = int(date_str[-4:-2])
        year = int(date_str[:-4])+1911
        DATE = pd.to_datetime(f"{year}-{month}-{date}",format='%Y-%m-%d',errors='coerce')

        return DATE.date()
    
    except ValueError:
        return math.nan


In [249]:
data112['交易年月日'] = data112['交易年月日'].apply(lambda x : transfor_date(str(x)) )

In [250]:
print(data112.head())

  鄉鎮市區       交易標的                土地位置建物門牌  土地移轉總面積平方公尺 都市土地使用分區 非都市土地使用分區  \
0  士林區  房地(土地+建物)          臺北市士林區大西路７４巷２號        21.67        商       NaN   
1  士林區  房地(土地+建物)  臺北市士林區延平北路八段２巷５５弄２０號二樓        29.67        住       NaN   
2  士林區  房地(土地+建物)          臺北市士林區中山北路七段９號        20.92        商       NaN   
3  士林區  房地(土地+建物)        臺北市士林區中山北路七段９號二樓        31.69        商       NaN   
4  士林區         車位      臺北市士林區士東路２００巷６７之１號         1.97        住       NaN   

   非都市土地使用編定       交易年月日      交易筆棟數 移轉層次 總樓層數           建物型態   主要用途    主要建材  \
0        NaN  2023-02-04  土地1建物1車位0   三層   四層   公寓(5樓含以下無電梯)    住家用  鋼筋混凝土造   
1        NaN  2023-02-03  土地1建物1車位0   二層   三層   公寓(5樓含以下無電梯)    住家用    加強磚造   
2        NaN  2023-02-04  土地1建物1車位0   一層   七層  華廈(10層含以下有電梯)    商業用  鋼筋混凝土造   
3        NaN  2023-02-04  土地3建物1車位0   二層   七層  華廈(10層含以下有電梯)    住家用  鋼筋混凝土造   
4        NaN  2023-02-02  土地2建物1車位2   一層  十二層             其他  一般零售業  鋼筋混凝土造   

     建築完成年月  建物移轉總面積平方公尺  建物現況格局.房  建物現況格局.廳  建物現況格局.衛 建物現況格局.

In [251]:
import math

data112['建築完成年月']= data112['建築完成年月'].apply(lambda x: transfor_date(str(x)))

In [252]:
data112['交易年月日']=pd.to_datetime(data112['交易年月日'], errors='coerce')
data112['建築完成年月']=pd.to_datetime(data112['建築完成年月'], errors='coerce')



In [253]:
data112['屋齡'] = (data112['交易年月日']-data112['建築完成年月']).dt.days


In [254]:
# Replace negative values with 0

data112['屋齡'] = data112['屋齡'].apply(lambda x: max(x, 0))


In [255]:
data112['屋齡'] = data112['屋齡']/365

In [256]:
data112['是否為預購屋'] = data112['屋齡'].apply(lambda x : '1' if x==0 else '0' )

In [257]:
import re
import matplotlib.pyplot as plt

In [258]:
data112.head()

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局.房,建物現況格局.廳,建物現況格局.衛,建物現況格局.隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,m,n,經度,緯度,單價(萬/坪),土地移轉總面積(坪),建物移轉總面積(坪),車位移轉總面積(坪),主建物面積(坪),附屬建物面積(坪),陽台面積(坪),車位總價元(萬),總價元(萬),屋齡,是否為預購屋
0,士林區,房地(土地+建物),臺北市士林區大西路７４巷２號,21.67,商,NaN,NaN,2023-02-04,土地1建物1車位0,三層,四層,公寓(5樓含以下無電梯),住家用,鋼筋混凝土造,1972-03-07,57.17,2,2,1,有,無,8520000,149029.0,NaN,0.00,0,親友、員工、共有人或其他特殊關係間之交易；,RPXNMLOJKHIGFEA97DA,57.17,0.00,0.00,無,NaN,112,2,25.105582,121.486351,49.254085,6.556732,17.298033,0.000000,17.298033,0.00000,0.000000,0.0,852.0,50.947945,0
1,士林區,房地(土地+建物),臺北市士林區延平北路八段２巷５５弄２０號二樓,29.67,住,NaN,NaN,2023-02-03,土地1建物1車位0,二層,三層,公寓(5樓含以下無電梯),住家用,加強磚造,1962-06-30,69.98,2,1,1,無,無,7800000,111460.0,NaN,0.00,0,NaN,RPPNMLPKKHIGFEA47EA,69.98,0.00,0.00,無,15.0,112,2,25.111533,121.536505,36.837530,8.977307,21.173979,0.000000,21.173979,0.00000,0.000000,0.0,780.0,60.638356,0
2,士林區,房地(土地+建物),臺北市士林區中山北路七段９號,20.92,商,NaN,NaN,2023-02-04,土地1建物1車位0,一層,七層,華廈(10層含以下有電梯),商業用,鋼筋混凝土造,1988-03-14,104.53,1,1,1,有,有,44000000,420932.0,NaN,0.00,0,壹樓空地。,RPUPMLKKKHIGFEA57DA,77.63,7.03,0.00,有,23.0,112,2,25.110783,121.534592,139.118026,6.329803,31.627837,0.000000,23.488654,2.12708,0.000000,0.0,4400.0,34.917808,0
3,士林區,房地(土地+建物),臺北市士林區中山北路七段９號二樓,31.69,商,NaN,NaN,2023-02-04,土地3建物1車位0,二層,七層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,1988-03-14,162.43,2,1,2,有,有,31000000,190851.0,NaN,0.00,0,陽台外推；,RPVPMLKKKHIGFEA67DA,142.80,0.00,11.36,有,24.0,112,2,25.080679,121.510582,63.076256,9.588502,49.146747,0.000000,43.207262,0.00000,3.437216,0.0,3100.0,34.917808,0
4,士林區,車位,臺北市士林區士東路２００巷６７之１號,1.97,住,NaN,NaN,2023-02-02,土地2建物1車位2,一層,十二層,其他,一般零售業,鋼筋混凝土造,2009-07-20,33.34,0,0,0,無,有,2300000,NaN,坡道機械,33.34,0,僅車位交易；,RPQNMLPKKHIGFEA47DA,0.00,0.00,0.00,有,25.0,112,2,25.103002,121.527599,NaN,0.596067,10.087746,10.087746,0.000000,0.00000,0.000000,0.0,230.0,13.547945,0


## 轉換交易層數資料

   1、 轉換購買總樓層數為數量

   2、轉換移轉層數資料，並且觀察是否有買多層樓的房屋，若有則會轉為是否購買一層以上的房屋
   
   已知購買全棟是為透天厝 


In [259]:
from chinesenumber import NumberParser
parser = NumberParser()

總樓層數移轉

In [260]:
data112['總樓層數'] = data112['總樓層數'].apply(lambda  x : 
                                  float(parser.numberify(str(x).split('層')[0])))

In [261]:
data112['移轉層次'].head(50)

0        三層
1        二層
2        一層
3        二層
4        一層
5       地下層
6         全
7        八層
8        一層
9        三層
10       二層
11     地下二層
12       四層
13       二層
14       一層
15     地下一層
16       二層
17       五層
18      十一層
19     地下二層
20      十三層
21       五層
22    一層，二層
23       六層
24       四層
25       六層
26     地下三層
27       二層
28       二層
29       一層
30       五層
31    四層，陽臺
32        全
33       三層
34       三層
35       一層
36       四層
37       三層
38       一層
39       三層
40       三層
41       一層
42       四層
43       四層
44       一層
45       二層
46       三層
47       五層
48       三層
49       六層
Name: 移轉層次, dtype: object

In [262]:
import re

def identify_mutiple_floors(x):

    # Convert input to string for processing
    x = str(x)

    x = parser.numberify(x)
    x = x.split('層')

    x = [re.sub(r'\D', '', word) for word in x]
    x = [word for word in x if word != '']

    

    return len(x)+1


In [263]:
floor = data112['移轉層次'].apply(lambda x : identify_mutiple_floors(x))
max_floor = data112['移轉層次'].apply(lambda x : identify_mutiple_floors(x)).idxmax()

print(data112['移轉層次'].apply(lambda x : identify_mutiple_floors(x)).max())

data112['移轉層次'].iloc[max_floor]

12


'一層，二層，三層，四層，五層，六層，七層，八層，九層，十層，十一層，陽台，騎樓，地下層'

In [264]:

# set definition fuinction for 

def numeric_transfor(floor,number):
    
    try:
        
        if floor == '地下層':
            
            return  -1
                     
        floor = str(floor).split('層')[number -1 ]        
        
        
        if floor == '全':    
            return 1
        
        # data.split('層') => ['二層', ] , the length of list is 2 
                     
        if re.search(r'地下',floor):
            x = floor.replace(r'地下','')
            x = x.replace(r'，','')
            x = - int(parser.numberify(x))
            return x
        
        else:
            x = floor.replace(r'，','')
            x = int(parser.numberify(x))
            return x
        

    except Exception as e :
        
        return floor


In [265]:
pd.set_option("display.max_rows",None)
data112['移轉層次'].apply(lambda x : str(x).split('層'))

0                                                    [三, ]
1                                                    [二, ]
2                                                    [一, ]
3                                                    [二, ]
4                                                    [一, ]
5                                                   [地下, ]
6                                                      [全]
7                                                    [八, ]
8                                                    [一, ]
9                                                    [三, ]
10                                                   [二, ]
11                                                 [地下二, ]
12                                                   [四, ]
13                                                   [二, ]
14                                                   [一, ]
15                                                 [地下一, ]
16                                                   [二,

In [266]:
data112['第一筆樓層數'] = data112['移轉層次'].apply(lambda x : numeric_transfor(x,1))

In [267]:
pd.set_option("display.max_rows",None)
data112['第一筆樓層數'].head(1000)

0            3
1            2
2            1
3            2
4            1
5           -1
6            1
7            8
8            1
9            3
10           2
11          -2
12           4
13           2
14           1
15          -1
16           2
17           5
18          11
19          -2
20          13
21           5
22           1
23           6
24           4
25           6
26          -3
27           2
28           2
29           1
30           5
31           4
32           1
33           3
34           3
35           1
36           4
37           3
38           1
39           3
40           3
41           1
42           4
43           4
44           1
45           2
46           3
47           5
48           3
49           6
50           3
51           3
52           6
53           5
54           2
55          12
56           2
57           6
58           5
59           2
60           3
61           2
62           2
63           6
64           3
65           4
66        

In [268]:
data112['第二筆層數'] = data112['移轉層次'].apply(lambda x : numeric_transfor(x,2))


# Count empty rows (where 'Content' is empty or whitespace)
empty_rows = data112[data112['第二筆層數'].str.strip() == ""].shape[0]

print(empty_rows/data112.shape[0])

0.8617532272590813


In [269]:
data112['第二筆層數']

0                   
1                   
2                   
3                   
4                   
5                 -1
6                  全
7                   
8                   
9                   
10                  
11                  
12                  
13                  
14                  
15                  
16                  
17                  
18                  
19                  
20                  
21                  
22                 2
23                  
24                  
25                  
26                  
27                  
28                  
29                  
30                  
31               ，陽臺
32                 全
33                  
34                  
35                  
36                  
37                  
38                  
39                  
40                  
41                  
42                  
43                  
44                  
45                  
46                  
47           

計算移轉層數為(全)的建物型態

In [270]:
print(f'移轉層數為全的層數筆數 : {data112.query('(移轉層次 == "全")').shape[0]}')
print(f'移轉層數為全以及建物型態為透天厝交集筆數 : {data112.query('(移轉層次 == "全") & (建物型態 == "透天厝")').shape[0]}')

移轉層數為全的層數筆數 : 500
移轉層數為全以及建物型態為透天厝交集筆數 : 500


# 定義是否購買多層數

In [271]:
import re

def identify_mutiple_floors(x):

    # Convert input to string for processing
    x = str(x)

    x = parser.numberify(x)
    x = x.split('層')

    x = [re.sub(r'\D', '', word) for word in x]
    x = [word for word in x if word != '']

    

    if len(x) > 1 :  
        return '1'  
    else:
        return '0'  


In [272]:
data112['是否夠買一層以上的房子']= data112['移轉層次'].apply(lambda x:identify_mutiple_floors(x))  

## 是否購買夾層屋

In [273]:
data112['購買是否有買夾層屋'] = data112['移轉層次'].apply(lambda x : '1' 
                                       if isinstance(x,str) and re.search(r'夾層',x) else '0')

## 公設面積包含電梯或樓梯

In [274]:
data112['公設面積包含電梯或樓梯'] = data112['移轉層次'].apply(lambda x : '1' 
                                         if isinstance(x,str) and re.search(r'電梯樓梯',x) else '0')

## 公設面積包含屋頂突出物

In [275]:
data112['公設面積包含屋頂突出物'] = data112['移轉層次'].apply(lambda x : '1'
                                          if isinstance(x,str) and re.search(r'屋頂突出物',x) else '0')

## 附屬面積含平台

In [276]:
data112['附屬面積含平台'] = data112['移轉層次'].apply(lambda x : '1' 
                                     if isinstance(x,str) and re.search(r'平台',x) else '0')

## 附屬面積含露台

In [277]:
data112['附屬面積含露台'] = data112['移轉層次'].apply(lambda x : '1' 
                                     if isinstance(x,str) and re.search(r'露台',x) else '0')

## 購買時是否有騎樓

In [278]:
data112['購買時是否有騎樓'] = data112['移轉層次'].apply(lambda x : '1' 
                                      if isinstance(x,str) and re.search(r'騎樓',x) else '0')

## 是否有購地下樓層

In [279]:
data112['是否有購地下樓層'] = data112['移轉層次'].apply(lambda x : '1' 
                                      if isinstance(x,str) and re.search(r'地下',x) else '0')

## 主要用途\

In [280]:
pd.set_option("display.max_row",None)

print(data112['主要用途'].value_counts())

主要用途
住家用                   14634
商業用                    2401
其他                      565
辦公用                     536
停車空間                    212
工業用                     201
見使用執照                   166
住商用                     136
一般事務所                   126
防空避難室兼停車場                32
一般零售業                    30
一般零售業甲組                  21
一般零售業（甲組）                18
日常服務業                    14
工商用                      13
農業用                      12
日常用品零售業                  10
策略性產業〈２８組資訊服務業〉           7
住工用                       7
自由職業事務所                   6
一般零售業甲組（不含日用百貨）           6
一般零售業〈甲組〉                 6
日常用品零售業不含便利商店             6
一般事務所（Ｇ２）                 5
停車場、防空避難室兼停車場             4
多戶住宅                      4
集合住宅                      3
一般事務所（Ｇ３）                 3
防空避難室、停車場                 3
一般零售業乙組                   3
一般零售業﹝甲組﹞                 3
停車位                       2
商辦用                       2
飲食業                       2
防空避難室兼商場、停車場              2
一般事務所（Ｇ２）辦公室   

In [281]:
print(data112['交易標的'].value_counts())

交易標的
房地(土地+建物)       13643
房地(土地+建物)+車位     5174
車位               1022
建物                147
Name: count, dtype: int64


In [282]:
print(f"房地主要用途遺失值:{data112.query("(交易標的 == '房地(土地+建物)')")['主要用途'].isna().sum()}")
print(f"房地加車位主要用途遺失值:{data112.query("(交易標的 == '房地(土地+建物)+車位')")['主要用途'].isna().sum()}")
print(f"車位主要用途遺失值:{data112.query("(交易標的 == '車位')")['主要用途'].isna().sum()}")
print(f"車位主要用途遺失值:{data112.query("(交易標的 == '建物')")['主要用途'].isna().sum()}")
print(f"資料主要用途遺失值:{data112['主要用途'].isna().sum()}")

房地主要用途遺失值:546
房地加車位主要用途遺失值:1
車位主要用途遺失值:183
車位主要用途遺失值:19
資料主要用途遺失值:749


In [283]:
def transfor_purpose(x):

    if isinstance(x,str):

        if re.search(r"零售",x) and re.search(r"停車",x):

            return '零售用途及停車用途'

        elif re.search(r"零售",x) :

            return '零售用途'

        elif re.search(r"停車",x):

            return '停車用途'

        elif re.search(r'事務',x):
            return '事務所'

        else :
            return x 
    else:
        return x

In [284]:
data112['主要用途'] = data112['主要用途'].apply(lambda x : transfor_purpose(x))

In [285]:
print(data112['主要用途'].value_counts())

主要用途
住家用                14634
商業用                 2401
其他                   565
辦公用                  536
停車用途                 275
工業用                  201
見使用執照                166
事務所                  145
住商用                  136
零售用途                 109
日常服務業                 14
工商用                   13
農業用                   12
住工用                    7
策略性產業〈２８組資訊服務業〉        7
多戶住宅                   4
集合住宅                   3
商辦用                    2
零售用途及停車用途              2
飲食業                    2
日常服務業〈不含理髮及美容業〉        1
管理員室                   1
策略性產業（資訊服務業）           1
Name: count, dtype: int64


## 都市土地使用轉換 對公共設施用地

In [286]:
def transfor_urban(x):

    if  isinstance(x,str):

        if re.search("公共設施用地",x) :

            return '公共設施用途'

        else :

            return x

    else :

        return x

In [287]:
data112['都市土地使用分區'] = data112['都市土地使用分區'].apply(lambda x : transfor_urban(x))

In [288]:
data112['都市土地使用分區'].value_counts()

都市土地使用分區
住                                                                                      11347
商                                                                                       6389
工                                                                                        391
都市：其他:第三種住宅區。                                                                            302
公共設施用途                                                                                   160
都市：其他:住宅用地(民生社區特定專用區)。                                                                   154
都市：其他:特定觀光商業專用區。                                                                         126
都市：其他:特定業務區(信義計畫特定專用區)。                                                                   91
都市：其他:特定專用區(供一般商業使用)。                                                                     51
都市：其他:特定休閒旅館住宅專用區。                                                                        46
都市：其他:工商混合區(不得作住宅使用)。                                        

## 轉換土地比例 (主建物占比、附屬建物占比、車位占比、公設比)

  . 建物移轉面積算進主建物面積、車位面積、陽台面積、附屬面積以及公設面積

  . 主建物占比 = (主建物面積)/(建物移轉面積 )

  . 陽台占比 = 陽台面積/(建物移轉總面積)

  . 附屬建物占比 = 附屬建物面積/(建物移轉總面積 )

  . 車位佔比 = 車位面積 / 建物移轉面積

  . 公占比 = 1 - 主建物占比 - 陽台面積占比 - 附屬面積占比  https://labhouse.ebc.net.tw/flash/2329

  . 是否有把 陽台 、附屬建物面積 跟 公占比 列入 坪數價格 : (總價格 - 車位價格)/ (建物移轉總面積) = 每坪價格 ， 相等答 1

  . 附屬建物價格及公設可不列入實際價錢坪數及總價錢裡，因此若主建物占比接近100%則列為附屬建物跟公設不列入坪數範圍內
     https://www.moi.gov.tw/News_Content.aspx?n=2&s=13530#:~:text=%E4%BE%8B%E5%A6%82%EF%BC%8C%E4%BB%A5%E5%90%AB%E8%BB%8A%E4%BD%8D%E4%BA%A4,%E6%A0%BC%E8%B3%87%E8%A8%8A%E6%9B%B4%E5%8A%A0%E9%80%8F%E6%98%8E%E3%80%82



## 主建物比例

In [289]:
data112['主建物占比'] = data112['主建物面積(坪)']/data112['建物移轉總面積(坪)']

## 陽台面積占比

In [290]:
data112['陽台面積占比'] = data112['陽台面積(坪)']/data112['建物移轉總面積(坪)'] 

## 附屬面積占比 

In [291]:
data112['附屬面積占比'] = data112['附屬建物面積(坪)']/data112['建物移轉總面積(坪)'] 

## 公占比

In [292]:
data112['公佔比'] = 1 - data112['主建物占比'] - data112['陽台面積占比'] - data112['附屬面積占比']

## 附屬建物是否記在坪數價格裡面
   
  . 車位若沒有面積只有車位總價，那麼車位坪數價格則會被列為房屋實價價格   https://mlvr.land.moi.gov.tw/jsp/qa.jsp
  
  . 方法 : (總價 - 車位價格) / (建物轉移面積) 是否 小於 坪數價格，如果是的話則列為附屬建物或公設比不計坪數價格

  . 有記附屬面積為面積計 1 ,無記附屬面積為 0
    

In [293]:
def record_price_on_adscript_land(car_total_land,car_totel_price,building_total_price,builing_total_land,land_price):

    try:

        if car_total_land >= 0 and  car_totel_price >= 0:           

            return int((((building_total_price - car_totel_price )/
                         (builing_total_land)) >= land_price))

    except Exception as e:
        return '未擁有這些資料'




In [294]:

data112['是否有記附屬面積'] = data112.apply(lambda row : record_price_on_adscript_land(
row['車位總價元(萬)'],
row['車位移轉總面積(坪)'],
row['總價元(萬)'],
row['建物移轉總面積(坪)'],
row['單價(萬/坪)']
),axis=1)

   

In [295]:
data112['備註'].head()

0    親友、員工、共有人或其他特殊關係間之交易；
1                      NaN
2                    壹樓空地。
3                    陽台外推；
4                   僅車位交易；
Name: 備註, dtype: object

## 轉換 備註 資料
   
   . 違章建物法規 : https://law.moj.gov.tw/LawClass/LawAll.aspx?pcode=D0070124

   . 違建定義 : https://masterlaw.com.tw/%e9%81%95%e5%bb%ba%e4%b8%80%e5%ae%9a%e8%a6%81%e6%8b%86%e5%97%8e/ 、 https://www.hbhousing.com.tw/News/Detail.aspx?Num=5143


In [296]:
import re

# design "未登記建物" 、 公共設施 、增建、頂樓加蓋 、陽台外推以及增建 

def transforNOTE(text):
    
    if pd.isna(text):
        
        return '一般交易'
    
    if isinstance(text,str) and re.search(r"未登記建物",text) and re.search(r"公共設施",text) and re.search(r"特殊關係間之交易",text) and (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)) :
        return "透過特殊關係的交易、有未登記建物、公共設施以及違章建物"    


    elif  isinstance(text,str) and re.search(r"特殊關係間之交易",text) and re.search(r"未登記建物",text) and re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "透過特殊關係的交易、未登記建物及公共設施"
    
    elif  isinstance(text,str) and re.search(r"特殊關係間之交易",text) and re.search(r"未登記建物",text) and not re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "透過特殊關係的交易、未登記建物及違章建築"
    
    elif isinstance(text,str) and not re.search(r"未登記建物",text) and re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "透過特殊關係的交易、公共設施及違章建物"
    
    elif isinstance(text,str) and  re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物、公共設施及違章建物"


    elif isinstance(text,str) and not re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "公共設施及違章建物"
    
    elif isinstance(text,str) and not re.search(r"未登記建物",text) and  re.search(r"特殊關係間之交易",text) and not re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "特殊關係交易及違章建物"
    
    elif isinstance(text,str) and not re.search(r"未登記建物",text) and  re.search(r"特殊關係間之交易",text) and re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "特殊關係交易及公共設施"
    
    elif isinstance(text,str) and  re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and not re.search(r"公共設施",text) and  (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物及違章建物"
    
    elif isinstance(text,str) and  re.search(r"未登記建物",text) and not re.search(r"特殊關係間之交易",text) and  re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物及公共設施"
    
    elif isinstance(text,str) and re.search(r"未登記建物",text) and  re.search(r"特殊關係間之交易",text) and not re.search(r"公共設施",text) and not (re.search(r'頂樓加蓋',text) or re.search(r'露臺外推',text) or re.search(r'陽台外推',text) or re.search(r'增建',text)):
        return "未登記建物及特殊關係交易"
    
    elif isinstance(text,str) and re.search(r"未登記建物",text) :
        return "未登記建物"
    
    elif isinstance(text,str) and re.search(r"公共設施",text):
        return "公共設施"
    
    elif isinstance(text,str) and (re.search(r"頂樓加蓋", text) or re.search(r'露臺外推',text) or re.search(r"陽台外推", text) or re.search(r"增建",text)):
        return "違章建物"
    
    elif isinstance(text,str) and re.search(r"特殊關係間之交易",text):
        
        return "特殊關係交易"
    
    elif isinstance(text,str) and re.search(r"政府機關標讓售",text):
        
        return "政府機關標售"
    
    else : 
        
        return text
    
    



In [297]:
data112.head()

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局.房,建物現況格局.廳,建物現況格局.衛,建物現況格局.隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,m,n,經度,緯度,單價(萬/坪),土地移轉總面積(坪),建物移轉總面積(坪),車位移轉總面積(坪),主建物面積(坪),附屬建物面積(坪),陽台面積(坪),車位總價元(萬),總價元(萬),屋齡,是否為預購屋,第一筆樓層數,第二筆層數,是否夠買一層以上的房子,購買是否有買夾層屋,公設面積包含電梯或樓梯,公設面積包含屋頂突出物,附屬面積含平台,附屬面積含露台,購買時是否有騎樓,是否有購地下樓層,主建物占比,陽台面積占比,附屬面積占比,公佔比,是否有記附屬面積
0,士林區,房地(土地+建物),臺北市士林區大西路７４巷２號,21.67,商,NaN,NaN,2023-02-04,土地1建物1車位0,三層,4.0,公寓(5樓含以下無電梯),住家用,鋼筋混凝土造,1972-03-07,57.17,2,2,1,有,無,8520000,149029.0,NaN,0.00,0,親友、員工、共有人或其他特殊關係間之交易；,RPXNMLOJKHIGFEA97DA,57.17,0.00,0.00,無,NaN,112,2,25.105582,121.486351,49.254085,6.556732,17.298033,0.000000,17.298033,0.00000,0.000000,0.0,852.0,50.947945,0,3,,0,0,0,0,0,0,0,0,1.000000,0.000000,0.000000,0.000000,1
1,士林區,房地(土地+建物),臺北市士林區延平北路八段２巷５５弄２０號二樓,29.67,住,NaN,NaN,2023-02-03,土地1建物1車位0,二層,3.0,公寓(5樓含以下無電梯),住家用,加強磚造,1962-06-30,69.98,2,1,1,無,無,7800000,111460.0,NaN,0.00,0,NaN,RPPNMLPKKHIGFEA47EA,69.98,0.00,0.00,無,15.0,112,2,25.111533,121.536505,36.837530,8.977307,21.173979,0.000000,21.173979,0.00000,0.000000,0.0,780.0,60.638356,0,2,,0,0,0,0,0,0,0,0,1.000000,0.000000,0.000000,0.000000,1
2,士林區,房地(土地+建物),臺北市士林區中山北路七段９號,20.92,商,NaN,NaN,2023-02-04,土地1建物1車位0,一層,7.0,華廈(10層含以下有電梯),商業用,鋼筋混凝土造,1988-03-14,104.53,1,1,1,有,有,44000000,420932.0,NaN,0.00,0,壹樓空地。,RPUPMLKKKHIGFEA57DA,77.63,7.03,0.00,有,23.0,112,2,25.110783,121.534592,139.118026,6.329803,31.627837,0.000000,23.488654,2.12708,0.000000,0.0,4400.0,34.917808,0,1,,0,0,0,0,0,0,0,0,0.742658,0.000000,0.067253,0.190089,0
3,士林區,房地(土地+建物),臺北市士林區中山北路七段９號二樓,31.69,商,NaN,NaN,2023-02-04,土地3建物1車位0,二層,7.0,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,1988-03-14,162.43,2,1,2,有,有,31000000,190851.0,NaN,0.00,0,陽台外推；,RPVPMLKKKHIGFEA67DA,142.80,0.00,11.36,有,24.0,112,2,25.080679,121.510582,63.076256,9.588502,49.146747,0.000000,43.207262,0.00000,3.437216,0.0,3100.0,34.917808,0,2,,0,0,0,0,0,0,0,0,0.879148,0.069938,0.000000,0.050914,1
4,士林區,車位,臺北市士林區士東路２００巷６７之１號,1.97,住,NaN,NaN,2023-02-02,土地2建物1車位2,一層,12.0,其他,零售用途,鋼筋混凝土造,2009-07-20,33.34,0,0,0,無,有,2300000,NaN,坡道機械,33.34,0,僅車位交易；,RPQNMLPKKHIGFEA47DA,0.00,0.00,0.00,有,25.0,112,2,25.103002,121.527599,NaN,0.596067,10.087746,10.087746,0.000000,0.00000,0.000000,0.0,230.0,13.547945,0,1,,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,1.000000,0


In [298]:
data112['備註'] = data112['備註'].apply(lambda x : transforNOTE(x) )

In [299]:
print(data112['備註'].value_counts())

備註
違章建物                                                                                                                                                                            8561
一般交易                                                                                                                                                                            6525
特殊關係交易                                                                                                                                                                          2287
僅車位交易；                                                                                                                                                                           867
特殊關係交易及違章建物                                                                                                                                                                      392
夾層；                                                                                         

In [300]:
print(data112.head())

  鄉鎮市區       交易標的                土地位置建物門牌  土地移轉總面積平方公尺 都市土地使用分區 非都市土地使用分區  \
0  士林區  房地(土地+建物)          臺北市士林區大西路７４巷２號        21.67        商       NaN   
1  士林區  房地(土地+建物)  臺北市士林區延平北路八段２巷５５弄２０號二樓        29.67        住       NaN   
2  士林區  房地(土地+建物)          臺北市士林區中山北路七段９號        20.92        商       NaN   
3  士林區  房地(土地+建物)        臺北市士林區中山北路七段９號二樓        31.69        商       NaN   
4  士林區         車位      臺北市士林區士東路２００巷６７之１號         1.97        住       NaN   

   非都市土地使用編定      交易年月日      交易筆棟數 移轉層次  總樓層數           建物型態  主要用途    主要建材  \
0        NaN 2023-02-04  土地1建物1車位0   三層   4.0   公寓(5樓含以下無電梯)   住家用  鋼筋混凝土造   
1        NaN 2023-02-03  土地1建物1車位0   二層   3.0   公寓(5樓含以下無電梯)   住家用    加強磚造   
2        NaN 2023-02-04  土地1建物1車位0   一層   7.0  華廈(10層含以下有電梯)   商業用  鋼筋混凝土造   
3        NaN 2023-02-04  土地3建物1車位0   二層   7.0  華廈(10層含以下有電梯)   住家用  鋼筋混凝土造   
4        NaN 2023-02-02  土地2建物1車位2   一層  12.0             其他  零售用途  鋼筋混凝土造   

      建築完成年月  建物移轉總面積平方公尺  建物現況格局.房  建物現況格局.廳  建物現況格局.衛 建物現況格局.隔間 有無

In [301]:
new_data = data112[['鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積(坪)', '都市土地使用分區',

           '交易年月日', '交易筆棟數', '建物型態', '主要用途', '主要建材',

        '建物現況格局.房', '建物現況格局.廳', '建物現況格局.衛','建物現況格局.隔間', '有無管理組織',

          '車位類別', '電梯', 'm', 'n','單價(萬/坪)', '土地移轉總面積(坪)',

        '建物移轉總面積(坪)', '車位移轉總面積(坪)', '車位總價元(萬)','總價元(萬)', '主建物面積(坪)',

        '附屬建物面積(坪)', '陽台面積(坪)', '屋齡', '是否為預購屋','是否夠買一層以上的房子', '購買是否有買夾層屋',

        '公設面積包含電梯或樓梯', '公設面積包含屋頂突出物', '附屬面積含平台','附屬面積含露台', '購買時是否有騎樓', 

        '是否有購地下樓層', '主建物占比', '陽台面積占比','附屬面積占比', '公佔比', '是否有記附屬面積','備註','第一筆樓層數','第二筆層數']]



In [302]:
new_data.to_excel('C:\資料探勘\資料處理正式\complete_processing_data_112.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\ruizhong\AppData\Local\Temp\ipykernel_22820\3808095984.py:1: SyntaxWarning: invalid escape sequence '\c'
  new_data.to_excel('C:\資料探勘\資料處理正式\complete_processing_data_112.xlsx')
